In [3]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

# Vetor Data Tutorial

Resources
* https://openmaptiles.org/
* https://github.com/CrunchyData/pg_tileserv
* GeoServer

## Actinia Routes

* (GET, PUT, DELETE) /locations/{location_name}/mapsets/{mapset_name}/vector_layers 
* (GET, POST, DELETE) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}
* (GET) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/render
* (POST) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/sampling_async
* (POST) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/sampling_sync

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/basin_test/vector_layers" | jq

## View Vector Data

In [4]:
!ace location="nc_spm_08" mapset="basin_test" -v

['point_basin']


In [84]:
pc = !ace location="nc_spm_08" mapset="basin_test" script="./scripts/basin.sh" -d | jq -r -c
PC_BODY = pc[0]

In [85]:
str(PC_BODY)

'{"version":"1","list":[{"module":"g.region","id":"g.region_1804289383","inputs":[{"param":"n","value":"252702.4094174725"},{"param":"s","value":"229056.70015573464"},{"param":"e","value":"618028.8126522538"},{"param":"w","value":"598579.6858165322"},{"param":"align","value":"direction_3k_10m_d@basin_test"}]},{"module":"r.circle","id":"r.circle_1804289383","flags":"b","inputs":[{"param":"coordinates","value":"601225.5895524962,241014.7657618352"},{"param":"max","value":"200"}],"outputs":[{"param":"output","value":"circle"}]},{"module":"r.stream.basins","id":"r.stream.basins_1804289383","flags":"c","inputs":[{"param":"direction","value":"direction_3k_10m_d@basin_test"},{"param":"stream_rast","value":"circle"},{"param":"memory","value":"1500"}],"outputs":[{"param":"basins","value":"point_basin_cloud"}]},{"module":"r.to.vect","id":"r.to.vect_1804289383","inputs":[{"param":"input","value":"point_basin_cloud"},{"param":"type","value":"area"},{"param":"column","value":"value"}],"outputs":[{"

## Expot to GCS

In [59]:

!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs" -H "Content-Type: application/json" \
   -d $PC_BODY | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    46  100    37  100     9    129     31 --:--:-- --:--:-- --:--:--   161
{
  "message": "Internal Server Error"
}
curl: (3) bad range in URL position 7:
list:[module:g.region
      ^


In [92]:
url = "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs"
# url = "http://localhost:8088/api/v3/locations/nc_spm_08/processing_export"

pc = !ace location="nc_spm_08" mapset="basin_test" script="./scripts/basin.sh" -d | jq -r -c
PC_BODY = json.loads(pc[0])
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=PC_BODY,
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-04-29 18:25:36.845229', 'accept_timestamp': 1651256736.8452282, 'api_info': {'endpoint': 'asyncephemeralexportgcsresource', 'method': 'POST', 'path': '/api/v3/locations/nc_spm_08/processing_async_export_gcs', 'request_url': 'http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs'}, 'datetime': '2022-04-29 18:25:36.846469', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-7dd0b16e-dc82-4210-bbbc-65ad91d8c419', 'status': 'accepted', 'time_delta': 0.0012469291687011719, 'timestamp': 1651256736.8464692, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-7dd0b16e-dc82-4210-bbbc-65ad91d8c419'}, 'user_id': 'actinia-gdi'}


In [94]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7105  100  7105    0     0  19957      0 --:--:-- --:--:-- --:--:-- 19957
{
  "accept_datetime": "2022-04-29 18:25:36.845229",
  "accept_timestamp": 1651256736.8452282,
  "api_info": {
    "endpoint": "asyncephemeralexportgcsresource",
    "method": "POST",
    "path": "/api/v3/locations/nc_spm_08/processing_async_export_gcs",
    "request_url": "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs"
  },
  "datetime": "2022-04-29 18:25:39.655502",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "g.region_1804289383",
          "inputs": [
            {
              "param": "n",
              "value": "252702.4094174725"
            },
            {
              "param": "s",
              "value": "229056.

In [ ]:
# GRASS GeoServer

In [ ]:
# GRASS MBTiles

In [ ]:
# PostGIS VectorTiles